# # Answer Key to the Data Wrangling with Spark SQL Quiz
Este questionário usa o mesmo conjunto de dados e a maioria das mesmas perguntas do "Questionário - Organização de dados com frames de dados do Jupyter Notebook" anterior. Para este questionário, no entanto, use Spark SQL em vez de quadros de dados Spark.

Recursos úteis:
http://spark.apache.org/docs/latest/api/python/pyspark.sql.html

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession \
    .builder \
    .appName('Spark SQL quiz') \
    .getOrCreate()

In [3]:
user_log = spark.read.json("data/sparkify_log_small.json")
user_log.createOrReplaceTempView('table')

# Question 1

Which page did user id "" (empty string) NOT visit?

In [4]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [5]:
# SELECT distinct pages for the blank user and distinc pages for all users
# Right join the results to find pages that blank visitor did not visit
spark.sql("select * \
            from ( \
                select distinct page \
                from table \
                where userId =='') as user_pages \
            right join( \
                select distinct page \
                from table) as all_pages \
            on user_pages.page = all_pages.page \
            where user_pages.page is null").show()

+----+----------------+
|page|            page|
+----+----------------+
|null|Submit Downgrade|
|null|       Downgrade|
|null|          Logout|
|null|   Save Settings|
|null|        Settings|
|null|        NextSong|
|null|         Upgrade|
|null|           Error|
|null|  Submit Upgrade|
+----+----------------+



# Question 2 - Reflect

Por que você pode preferir usar SQL em vez de frames de dados? Por que você pode preferir frames de dados em vez de SQL?

Tanto Spark SQL quanto Spark Data Frames fazem parte da biblioteca Spark SQL. Portanto, ambos usam o Spark SQL Catalyst Optimizer para otimizar as consultas.

Você pode preferir SQL em vez de frames de dados porque a sintaxe é mais clara, especialmente para equipes já experientes em SQL.

Os frames de dados do Spark oferecem mais controle. Você pode dividir suas consultas em etapas menores, o que pode tornar a depuração mais fácil. Você também pode [armazenar em cache] (https://unraveldata.com/to-cache-or-not-to-cache/) resultados intermediários ou [reparticionar] (https://hackernoon.com/managing-spark-partitions-with -coalesce-and-repartition-4050c57ad5c4) resultados intermediários.

# Question 3

Quantas usuárias temos no conjunto de dados?

In [6]:
spark.sql("select count(distinct userId) \
            from table \
            where gender = 'F'").show()

+----------------------+
|count(DISTINCT userId)|
+----------------------+
|                   462|
+----------------------+



# Question 4

Quantas músicas do artista mais tocado foram tocadas?

In [7]:
# here is one solution
spark.sql("select Artist, count(artist) as plays \
    from table \
    group by Artist \
    order by plays desc \
    limit 1").show()

+--------+-----+
|  Artist|plays|
+--------+-----+
|Coldplay|   83|
+--------+-----+



In [8]:
# Here is an alternative solution
# Get the artist play counts
play_counts = spark.sql("SELECT Artist, COUNT(Artist) AS plays \
        FROM table \
        GROUP BY Artist")

# save the results in a new view
play_counts.createOrReplaceTempView("artist_counts")

# use a self join to find where the max play equals the count value
spark.sql("SELECT a2.Artist, a2.plays FROM \
          (SELECT max(plays) AS max_plays FROM artist_counts) AS a1 \
          JOIN artist_counts AS a2 \
          ON a1.max_plays = a2.plays \
          ").show()

+--------+-----+
|  Artist|plays|
+--------+-----+
|Coldplay|   83|
+--------+-----+



# Question 5 (challenge)

Quantas músicas os usuários ouvem em média entre as visitas à nossa página inicial? Arredonde sua resposta para o número inteiro mais próximo.

In [ ]:
# SELECT CASE WHEN 1 > 0 THEN 1 WHEN 2 > 0 THEN 2.0 ELSE 1.2 END;

is_home = spark.sql("SELECT userId, page, ts, CASE WHEN page = 'Home' \
                THEN 1 ELSE 0 END AS is_home FROM TABLE \
                where (page = 'NextSong') or (page = 'Home') \
                ")

# Keep the results in a new view
is_home.createOrReplaceTempView('is_home_table')

# find the cumulative sum over the is_home column
cumulative_sum = spark.sql("SELECT *, SUM(is_home) OVER \
            (PARTITION BY userId ORDER BY ts DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS period \
            FROM is_home_table")

# Keep the result in a View
cumulative_sum.createOrReplaceTempView("period_table")

# find the averange count for NextSong
spark.sql("SELECT AVG(count_results) FROM \
        (SELECT COUNT(*) AS count_results FROM period_table \
GROUP BY userId, period, page HAVING page = 'NestSong') AS counts").show()